In [1]:
import cv2
import numpy as np
import imageio as io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,StandardScaler
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
import skimage.io as io
import math
from operator import eq
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from skimage import color
from operator import eq
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import rbf_kernel
import math

In [2]:
le = preprocessing.LabelEncoder()
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)
sigma=0.0008
c=64
reg=100
err=1e-4

In [3]:
def DLTP(ic,arr1):
    Ic=ic
    arr=arr1[0]
    for i in range(len(arr)-1):        
        In=arr[i]
        tao=abs(In-Ic)/(Ic+0.0001)
        if(In>Ic+tao):
            arr[i]=1
        elif(In<=Ic-tao):
            arr[i]=-1
        else:
            arr[i]=0
    return arr        

In [4]:
def Pos_DLTP(arr):
    pos_arr=np.zeros((1,8), np.uint8)[0]
    k=0
    for i in arr:
        if (i==1):
            pos_arr[k]=1
        k=k+1    
    return  pos_arr       

In [5]:
def Neg_DLTP(arr):
    neg_arr=np.zeros((1,8), np.uint8)[0]
    k=0
    for i in arr:
        if (i==-1):
            neg_arr[k]=1
        k=k+1     
    return  neg_arr       

In [6]:
def Convertion(arr):
    value=0
    arr_value=[1,2,4,8,16,32,64,128]
    value=np.dot(arr,arr_value)
    return value

In [7]:
def padding(block,size):
    padd=np.pad(block, (size, size), 'constant', constant_values=(0, 0))
    return padd

In [8]:
def GetArray(block,x,y):
    one=block[x-1][y+1]
    two=block[x][y+1]
    three=block[x+1][y+1]
    four=block[x+1][y]
    five=block[x-1][y+1]
    six=block[x+1][y-1]
    seven=block[x][y-1]
    eight=block[x-1][y-1]
    zero=block[x][y]
    arr=[]
    arr.append([one,two,three,four,five,six,seven,eight])
    return zero,arr

In [9]:
def DLTP_Hist(block):
    img=padding(block,1)
    w,l=img.shape
    tlbphist=[]
    neg_hist=[]
    pos_hist=[]
    for y in range(1,w-1):
        for x in range(1,l-1):
            gc,array=GetArray(img,x,y)
            array2=array1=DLTP(gc,array)
            neg_arr=Neg_DLTP(array1)
            pos_arr=Pos_DLTP(array2)
            neg_hist.append(Convertion(neg_arr))
            pos_hist.append(Convertion(pos_arr))
    (histpos, _) = np.histogram(np.array(pos_hist),bins=np.arange(0,255),range=(0,255))
    histpos = histpos.astype("float")
    (histneg, _) = np.histogram(np.array(neg_hist),bins=np.arange(0,255),range=(0,255))
    histneg = histneg.astype("float")
    return np.array(histneg).flatten(),np.array(histpos).flatten()

In [10]:
def Algorithm_DLTP(img1,blocksize=4):
    w,l=img1.shape
    window=w//blocksize
    Ndltp_hist=[]
    Pdltp_hist=[]
    img=padding(img1,2)
    for r in range(2,w-2,window):
        for c in range(2,l-2,window):
            block = img[r:r+window,c:c+window]
            hneg,hpos= DLTP_Hist(block)
            Pdltp_hist.append(np.array(hpos))
            Ndltp_hist.append(np.array(hneg))
    dltpfeature=[]        
    dltpfeature.append(np.array(Pdltp_hist).flatten())
    dltpfeature.append(np.array(Ndltp_hist).flatten())
    return np.array(dltpfeature).flatten()

In [11]:
# img  = io.imread('S010_006_00000013.png')
# f=feature_extract(img)
# print(np.array(f).shape )

In [12]:
def feature_DLTP(imgs):
    gray=preprocessing1(imgs)
    enhanced=global_contrast_normalization(gray,reg,err)
    norm=min_max_normalize(enhanced)
    return np.array(Algorithm_DLTP(norm,4))         

In [13]:
def preprocessing1(img):
    resized = cv2.resize(img,(48,48), interpolation = cv2.INTER_AREA)
    gray=color.rgb2gray(resized)
    return gray

In [14]:
def load_dataset():
    features = []
    labels = []
    for i, fn in enumerate(img_filenames):
            label = fn.split('.')[0]
            path = os.path.join(path_to_dataset, fn)
            subpath=os.listdir(path)
            for k, d in enumerate(subpath):             
                labels.append(label)
                imgpath = os.path.join(path, d)
                imgs = io.imread(imgpath)
                features.append(feature_DLTP(imgs))
                if k > 0 and k % 10== 0:
                    print("[INFO] processed {}/{}".format(k, len(subpath)))
    return features, labels            

In [15]:
def normalization(train_features, test_features):
    scaler = StandardScaler()
    scaler.fit(train_features)

    train_features = scaler.transform(train_features)
    test_features = scaler.transform(test_features)
    return train_features, test_features

In [16]:
def min_max_normalize(img):
    return ((img-min(img.flatten()))/(max(img.flatten())-min(img.flatten())))*255

In [17]:
def global_contrast_normalization(image1,reg,err):
    image=np.array(image1)
    mean=np.mean(image)
    image=(image-mean)/(max(err,math.sqrt((reg)+np.mean((image-mean)**2))))
    return image

In [18]:
def split_data(features,labels,precentage):
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    return train_features, test_features, train_labels, test_labels

In [19]:
def encode(labels):
    le = preprocessing.LabelEncoder()
    return le.fit_transform(labels)

In [20]:
# path_to_dataset =r'E:\datasetemotion\val\val'
# path_to_dataset =r'E:\datasetemotion\train\train'
# path_to_dataset =r'D:\Download\jaffedbase'
path_to_dataset =r'E:\emotion\archive\ck\CK+48'
img_filenames = os.listdir(path_to_dataset)


In [21]:
img_filenames = os.listdir(path_to_dataset)
print(img_filenames)

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']


In [22]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset()
print('Finished loading dataset.')

Loading dataset. This will take time ...
[INFO] processed 10/135
[INFO] processed 20/135
[INFO] processed 30/135
[INFO] processed 40/135
[INFO] processed 50/135
[INFO] processed 60/135
[INFO] processed 70/135
[INFO] processed 80/135
[INFO] processed 90/135
[INFO] processed 100/135
[INFO] processed 110/135
[INFO] processed 120/135
[INFO] processed 130/135
[INFO] processed 10/54
[INFO] processed 20/54
[INFO] processed 30/54
[INFO] processed 40/54
[INFO] processed 50/54
[INFO] processed 10/177
[INFO] processed 20/177
[INFO] processed 30/177
[INFO] processed 40/177
[INFO] processed 50/177
[INFO] processed 60/177
[INFO] processed 70/177
[INFO] processed 80/177
[INFO] processed 90/177
[INFO] processed 100/177
[INFO] processed 110/177
[INFO] processed 120/177
[INFO] processed 130/177
[INFO] processed 140/177
[INFO] processed 150/177
[INFO] processed 160/177
[INFO] processed 170/177
[INFO] processed 10/75
[INFO] processed 20/75
[INFO] processed 30/75
[INFO] processed 40/75
[INFO] processed 50/

In [23]:
train_features, test_features, train_labels, test_labels=split_data(np.array(features),np.array(labels),0.2)

In [24]:
print(np.array(train_features).shape)

(784, 8128)


In [25]:
#PCA

In [26]:
pca = PCA(n_components=0.7)
X_train1 = pca.fit_transform(train_features)
X_test1 = pca.transform(test_features)
train_features=X_train1
test_features=X_test1
train_labels=train_labels
test_labels=test_labels

In [27]:
print(np.array(train_features).shape)

(784, 14)


In [28]:
#K-ELM

In [29]:
def kernel(X, Y=None,g=None):
    k=rbf_kernel(X, Y, gamma=g)
    return k

In [30]:
def KELM_Beta(sigma,c,x,y):
    le.fit(y)
    y = le.transform(y)
    emotion = len(np.unique(y))
    One_Hot_Encoding=np.zeros((emotion,emotion), np.uint8)
    for em in range (emotion):
        One_Hot_Encoding[em][em]=1
    T = One_Hot_Encoding[y, :]    
    N,d=x.shape
    Omega=kernel(x,None,sigma)
    I=np.eye(N)
    Beta = np.linalg.inv((I /c) + Omega).dot(T)    
    return Beta,One_Hot_Encoding

In [31]:
def KELM_Output(sigma,x,x_test,Beta,One_Hot_Encoding):
    hx=kernel(x_test,x,sigma)
    fx= hx.dot(Beta)
    p=np.argmax(fx,axis=1)
    return p

In [32]:
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)    
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(y_pred)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc

In [33]:
Beta,One_Hot_Encoding=KELM_Beta(sigma,c,np.array(train_features),np.array(train_labels))
p1=KELM_Output(sigma,np.array(train_features),np.array(test_features),Beta,One_Hot_Encoding)
print("test",acc(p1,test_labels))

p2=KELM_Output(sigma,np.array(train_features),np.array(train_features),Beta,One_Hot_Encoding)
print("train",acc(p2,train_labels))

# test 97.46192893401016
# train 100.0


test 97.46192893401016
train 100.0


In [34]:
test_labels1=encode(test_labels)
print(accuracy_score(test_labels1, p1))
print(classification_report(test_labels1, p1))
print(confusion_matrix(test_labels1, p1))

0.9746192893401016
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       1.00      1.00      1.00         6
           2       1.00      0.90      0.95        41
           3       1.00      1.00      1.00        15
           4       0.93      1.00      0.96        39
           5       0.96      1.00      0.98        22
           6       1.00      0.98      0.99        54

    accuracy                           0.97       197
   macro avg       0.98      0.98      0.98       197
weighted avg       0.98      0.97      0.97       197

[[20  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0]
 [ 1  0 37  0  3  0  0]
 [ 0  0  0 15  0  0  0]
 [ 0  0  0  0 39  0  0]
 [ 0  0  0  0  0 22  0]
 [ 0  0  0  0  0  1 53]]


In [35]:
# test 52.79187817258884
# train 88.13775510204081
#after normalization
# test 53.299492385786806
# train 89.54081632653062
# test 82.74111675126903
# train 100.0
# test 97.46192893401016
# train 100.0


In [36]:
#KNN 

In [37]:
print(np.array(train_features).shape)

(784, 14)


In [38]:
def KNN_classifier(kcount,train_features, train_labels):
    classifier = KNeighborsClassifier(n_neighbors=kcount)
    classifier.fit(list(train_features),list(train_labels) )
    return classifier

classifier=KNN_classifier(3,train_features, train_labels)
y_pred = classifier.predict(test_features)
accuracy = classifier.score(test_features, test_labels)
print('Test accuracy:', accuracy*100, '%')
accuracyt = classifier.score(train_features, train_labels)
print('Train accuracy:', accuracyt*100, '%')

# Test accuracy: 78.68020304568529 %
# Train accuracy: 97.5765306122449 %

Test accuracy: 78.68020304568529 %
Train accuracy: 97.5765306122449 %


In [39]:
# print(test_labels.flatten())
print(accuracy_score(test_labels, y_pred))
print(classification_report(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred))

0.7868020304568528
              precision    recall  f1-score   support

       anger       0.57      0.80      0.67        20
    contempt       0.86      1.00      0.92         6
     disgust       0.72      0.63      0.68        41
        fear       0.93      0.87      0.90        15
       happy       0.76      0.82      0.79        39
     sadness       0.74      0.64      0.68        22
    surprise       0.94      0.89      0.91        54

    accuracy                           0.79       197
   macro avg       0.79      0.81      0.79       197
weighted avg       0.80      0.79      0.79       197

[[16  0  0  0  2  2  0]
 [ 0  6  0  0  0  0  0]
 [ 5  0 26  0  7  2  1]
 [ 0  0  2 13  0  0  0]
 [ 2  0  3  0 32  0  2]
 [ 5  1  2  0  0 14  0]
 [ 0  0  3  1  1  1 48]]


In [40]:
#SVM

In [41]:
#RBF kernel will be suitable for complex
#scale--->gamma 1/(n+std)
k='rbf'
gamma = 'scale'
c=900
clf = SVC(C=c,kernel=k,gamma=gamma)
clf.fit(train_features, train_labels)
print ('Test score', clf.score(test_features, test_labels)*100 )
print ('Train score', clf.score(train_features, train_labels)*100 )
# Test score 43.14720812182741
# Train score 52.67857142857143
# Test score 95.93908629441624
# Train score 100.0

Test score 95.93908629441624
Train score 100.0


In [42]:
#save model

In [353]:
import joblib
# joblib.dump(p, 'model.pkl')
